# Step 3: Information Extraction Using Agentic Methods


This notebook performs information extraction from classified document sections using AWS Bedrock.

This extraction method utilises Strands Agent for a self correcting mechanism working against pydantic models internally to ensure schema adherence.

**Important**

This method only works with higher intelligence models that support tool use. Recommended model family is Anthropic Claude.
OpenAI models should perform well too but they are limited to text input only inside bedrock.
For Amazon models you should use Nova Premier.

**Inputs:**
- Document object with classification results from Step 2
- Extraction configuration
- Document classes with attributes definition

**Outputs:**
- Document with extraction results for each section
- Structured data extracted based on document class attributes

## 1. Load Previous Step Data

In [ ]:
ROOTDIR="../.."
%pip install -q -e "{ROOTDIR}/lib/idp_common_pkg[agents]"

In [ ]:
import json
import logging
import os
import time
from pathlib import Path

import boto3
from idp_common import extraction

# Import IDP libraries
from idp_common.models import Document, Status

# Configure logging
logging.basicConfig(level=logging.WARNING)
logging.getLogger('idp_common.extraction').setLevel(logging.INFO)
logging.getLogger('idp_common.bedrock.client').setLevel(logging.INFO)

print("Libraries imported successfully")

In [ ]:
# Load document from previous step
classification_data_dir = Path(".data/step2_classification")

# Load document object from JSON
document_path = classification_data_dir / "document.json"
with open(document_path, 'r') as f:
    document = Document.from_json(f.read())

# Load configuration directly from config files
import yaml

config_dir = Path("config")
CONFIG = {}

# Load each configuration file
config_files = [
    "extraction.yaml",
    "classes.yaml"
]

for config_file in config_files:
    config_path = config_dir / config_file
    if config_path.exists():
        with open(config_path, 'r') as f:
            file_config = yaml.safe_load(f)
            CONFIG.update(file_config)
        print(f"Loaded {config_file}")
    else:
        print(f"Warning: {config_file} not found")

# Load environment info
env_path = classification_data_dir / "environment.json"
with open(env_path, 'r') as f:
    env_info = json.load(f)

# Set environment variables
os.environ['AWS_REGION'] = env_info['region']
os.environ['METRIC_NAMESPACE'] = 'IDP-Modular-Pipeline'

print(f"Loaded document: {document.id}")
print(f"Document status: {document.status.value}")
print(f"Number of sections: {len(document.sections) if document.sections else 0}")
print(f"Loaded configuration sections: {list(CONFIG.keys())}")

## Agentic Extraction Implementation

### 2. Configure Extraction Service - with Agentic

In [ ]:
# Extract extraction configuration
CONFIG["extraction"]["agentic"] = {"enabled":True}
# Using Sonnet model - recommended for agentic extraction
CONFIG["extraction"]["model"] = "us.anthropic.claude-sonnet-4-20250514-v1:0"
# Alternative: CONFIG["extraction"]["model"] = "us.anthropic.claude-sonnet-4-20250514-v1:0"


extraction_config = CONFIG.get('extraction', {})
print("Extraction Configuration:")
print(f"Model: {extraction_config.get('model')}")
print(f"Temperature: {extraction_config.get('temperature')}")
print(f"Max Tokens: {extraction_config.get('max_tokens')}")
print("*"*50)

print(f"System Prompt:\n{extraction_config.get('system_prompt')}")
print("*"*50)
print(f"Task Prompt:\n{extraction_config.get('task_prompt')}")
print("*"*50)

# Display available document classes and their attributes
classes = CONFIG.get('classes', [])
print("\nDocument Classes and Attributes:")
for cls in classes:
    print(f"\n{cls['$id']} ({len(cls.get('attributes', []))} attributes):")
    for attr in cls.get('attributes', [])[:3]:  # Show first 3 attributes
        print(f"  - {attr['$id']}: {attr['description'][:100]}...")
    if len(cls.get('attributes', [])) > 3:
        print(f"  ... and {len(cls.get('attributes', [])) - 3} more")

In [ ]:
# Create extraction service with Bedrock
extraction_service = extraction.ExtractionService(config=CONFIG)

print("Extraction service initialized")

### 3. Extract Information from Document Sections

In [ ]:
# Helper function to parse S3 URIs and load JSON
def parse_s3_uri(uri):
    parts = uri.replace("s3://", "").split("/")
    bucket = parts[0]
    key = "/".join(parts[1:])
    return bucket, key

def load_json_from_s3(uri):
    s3_client = boto3.client('s3')
    bucket, key = parse_s3_uri(uri)
    response = s3_client.get_object(Bucket=bucket, Key=key)
    content = response['Body'].read().decode('utf-8')
    return json.loads(content)

print("Helper functions defined")

In [ ]:
print("Extracting information from document sections...")

if not document.sections:
    print("No sections found in document. Cannot proceed with extraction.")
else:
    extraction_results = []
    
    # Process each section (limit to first 3 to save time in demo)
    n = min(3, len(document.sections))
    print(f"Processing first {n} of {len(document.sections)} sections...")
    
    for i, section in enumerate(document.sections[:n]):
        print(f"\n--- Processing Section {i+1}/{n} ---")
        print(f"Section ID: {section.section_id}")
        print(f"Classification: {section.classification}")
        print(f"Pages: {section.page_ids}")
        
        # Process section extraction
        start_time = time.time()
        document = extraction_service.process_document_section(
            document=document,
            section_id=section.section_id
        )
        extraction_time = time.time() - start_time
        
        print(f"Extraction completed in {extraction_time:.2f} seconds")
        
        # Record results
        extraction_results.append({
            'section_id': section.section_id,
            'classification': section.classification,
            'processing_time': extraction_time,
            'extraction_result_uri': getattr(section, 'extraction_result_uri', None)
        })
    
    print(f"\nExtraction complete for {n} sections.")

### 4. Display Extraction Results

In [ ]:
print("\n=== Extraction Results ===")

if document.sections:
    for i, section in enumerate(document.sections[:n]):
        print(f"\n--- Section {section.section_id} ({section.classification}) ---")
        
        if hasattr(section, 'extraction_result_uri') and section.extraction_result_uri:
            try:
                # Load extraction results from S3
                extraction_data = load_json_from_s3(section.extraction_result_uri)
                
                print(f"Extraction Result URI: {section.extraction_result_uri}")
                
                # Display inference results
                if 'inference_result' in extraction_data:
                    inference_result = extraction_data['inference_result']
                    print("Extracted Data:")
                    for attr_name, attr_value in inference_result.items():
                        if attr_value is not None:
                            # Truncate long values for display
                            display_value = str(attr_value)[:1000] + "..." if len(str(attr_value)) > 1000 else attr_value
                            print(f"  {attr_name}: {display_value}")
                        else:
                            print(f"  {attr_name}: null")
                else:
                    print("No inference results found")
                    
                # Display metadata if available
                if 'metadata' in extraction_data:
                    metadata = extraction_data['metadata']
                    print(f"Processing time: {metadata.get('extraction_time_seconds', 'N/A')} seconds")
                    
            except Exception as e:
                print(f"Error loading extraction results: {e}")
        else:
            print("No extraction results available")
else:
    print("No sections to display")

### 5. Save Results for Next Step

In [ ]:
# Create data directory for this step
data_dir = Path(".data/step3_extraction")
data_dir.mkdir(parents=True, exist_ok=True)

# Save updated document object as JSON
document_path = data_dir / "document.json"
with open(document_path, 'w') as f:
    f.write(document.to_json())

# Save configuration (pass through)
config_path = data_dir / "config.json"
with open(config_path, 'w') as f:
    json.dump(CONFIG, f, indent=2)

# Save environment info (pass through)
env_path = data_dir / "environment.json"
with open(env_path, 'w') as f:
    json.dump(env_info, f, indent=2)

# Save extraction-specific results summary
extraction_summary = {
    'model_used': extraction_config.get('model'),
    'sections_processed': len(extraction_results) if 'extraction_results' in locals() else 0,
    'total_sections': len(document.sections) if document.sections else 0,
    'section_results': extraction_results if 'extraction_results' in locals() else [],
    'sections_with_extractions': [
        {
            'section_id': section.section_id,
            'classification': section.classification,
            'extraction_result_uri': getattr(section, 'extraction_result_uri', None),
            'has_results': hasattr(section, 'extraction_result_uri') and section.extraction_result_uri is not None
        } for section in (document.sections or [])
    ]
}

extraction_summary_path = data_dir / "extraction_summary.json"
with open(extraction_summary_path, 'w') as f:
    json.dump(extraction_summary, f, indent=2)

print(f"Saved document to: {document_path}")
print(f"Saved configuration to: {config_path}")
print(f"Saved environment info to: {env_path}")
print(f"Saved extraction summary to: {extraction_summary_path}")

### 6. Summary

In [ ]:
sections_processed = len(extraction_results) if 'extraction_results' in locals() else 0
sections_with_results = sum(1 for section in (document.sections or []) if hasattr(section, 'extraction_result_uri') and section.extraction_result_uri)

print("=== Step 3: Extraction Complete ===")
print(f"✅ Document processed: {document.id}")
print(f"✅ Sections processed: {sections_processed} of {len(document.sections) if document.sections else 0}")
print(f"✅ Sections with results: {sections_with_results}")
print(f"✅ Model used: {extraction_config.get('model')}")
print("✅ Data saved to: .data/step3_extraction/")
print("\n📌 Next step: Run step4_assessment.ipynb")

## Traditional Extraction

### 2. Configure Extraction without Agentic

In [ ]:
# Extract extraction configuration
CONFIG["extraction"]["agentic"] = {"enabled":False}
# For traditional extraction, can use simpler models
CONFIG["extraction"]["model"] = "us.anthropic.claude-sonnet-4-20250514-v1:0"


extraction_config = CONFIG.get('extraction', {})
print("Extraction Configuration:")
print(f"Model: {extraction_config.get('model')}")
print(f"Temperature: {extraction_config.get('temperature')}")
print(f"Max Tokens: {extraction_config.get('max_tokens')}")
print("*"*50)

print(f"System Prompt:\n{extraction_config.get('system_prompt')}")
print("*"*50)
print(f"Task Prompt:\n{extraction_config.get('task_prompt')}")
print("*"*50)

# Display available document classes and their attributes
classes = CONFIG.get('classes', [])
print("\nDocument Classes and Attributes:")
for cls in classes:
    print(f"\n{cls['name']} ({len(cls.get('attributes', []))} attributes):")
    for attr in cls.get('attributes', [])[:3]:  # Show first 3 attributes
        print(f"  - {attr['name']}: {attr['description'][:100]}...")
    if len(cls.get('attributes', [])) > 3:
        print(f"  ... and {len(cls.get('attributes', [])) - 3} more")

In [ ]:
# Create extraction service with Bedrock
extraction_service = extraction.ExtractionService(config=CONFIG)

print("Extraction service initialized")

### 3. Extract Information from Document Sections

In [ ]:
# Helper function to parse S3 URIs and load JSON
def parse_s3_uri(uri):
    parts = uri.replace("s3://", "").split("/")
    bucket = parts[0]
    key = "/".join(parts[1:])
    return bucket, key

def load_json_from_s3(uri):
    s3_client = boto3.client('s3')
    bucket, key = parse_s3_uri(uri)
    response = s3_client.get_object(Bucket=bucket, Key=key)
    content = response['Body'].read().decode('utf-8')
    return json.loads(content)

print("Helper functions defined")

In [ ]:
print("Extracting information from document sections...")

if not document.sections:
    print("No sections found in document. Cannot proceed with extraction.")
else:
    extraction_results = []
    
    # Process each section (limit to first 3 to save time in demo)
    n = min(3, len(document.sections))
    print(f"Processing first {n} of {len(document.sections)} sections...")
    
    for i, section in enumerate(document.sections[:n]):
        print(f"\n--- Processing Section {i+1}/{n} ---")
        print(f"Section ID: {section.section_id}")
        print(f"Classification: {section.classification}")
        print(f"Pages: {section.page_ids}")
        
        # Process section extraction
        start_time = time.time()
        document = extraction_service.process_document_section(
            document=document,
            section_id=section.section_id
        )
        extraction_time = time.time() - start_time
        
        print(f"Extraction completed in {extraction_time:.2f} seconds")
        
        # Record results
        extraction_results.append({
            'section_id': section.section_id,
            'classification': section.classification,
            'processing_time': extraction_time,
            'extraction_result_uri': getattr(section, 'extraction_result_uri', None)
        })
    
    print(f"\nExtraction complete for {n} sections.")

### 4. Display Extraction Results

In [ ]:
print("\n=== Extraction Results ===")

if document.sections:
    for i, section in enumerate(document.sections[:n]):
        print(f"\n--- Section {section.section_id} ({section.classification}) ---")
        
        if hasattr(section, 'extraction_result_uri') and section.extraction_result_uri:
            try:
                # Load extraction results from S3
                extraction_data = load_json_from_s3(section.extraction_result_uri)
                
                print(f"Extraction Result URI: {section.extraction_result_uri}")
                
                # Display inference results
                if 'inference_result' in extraction_data:
                    inference_result = extraction_data['inference_result']
                    print("Extracted Data:")
                    for attr_name, attr_value in inference_result.items():
                        if attr_value is not None:
                            # Truncate long values for display
                            display_value = str(attr_value)[:1000] + "..." if len(str(attr_value)) > 1000 else attr_value
                            print(f"  {attr_name}: {display_value}")
                        else:
                            print(f"  {attr_name}: null")
                else:
                    print("No inference results found")
                    
                # Display metadata if available
                if 'metadata' in extraction_data:
                    metadata = extraction_data['metadata']
                    print(f"Processing time: {metadata.get('extraction_time_seconds', 'N/A')} seconds")
                    
            except Exception as e:
                print(f"Error loading extraction results: {e}")
        else:
            print("No extraction results available")
else:
    print("No sections to display")

### 5. Save Results for Next Step

In [ ]:
# Create data directory for this step
data_dir = Path(".data/step3_extraction")
data_dir.mkdir(parents=True, exist_ok=True)

# Save updated document object as JSON
document_path = data_dir / "document.json"
with open(document_path, 'w') as f:
    f.write(document.to_json())

# Save configuration (pass through)
config_path = data_dir / "config.json"
with open(config_path, 'w') as f:
    json.dump(CONFIG, f, indent=2)

# Save environment info (pass through)
env_path = data_dir / "environment.json"
with open(env_path, 'w') as f:
    json.dump(env_info, f, indent=2)

# Save extraction-specific results summary
extraction_summary = {
    'model_used': extraction_config.get('model'),
    'sections_processed': len(extraction_results) if 'extraction_results' in locals() else 0,
    'total_sections': len(document.sections) if document.sections else 0,
    'section_results': extraction_results if 'extraction_results' in locals() else [],
    'sections_with_extractions': [
        {
            'section_id': section.section_id,
            'classification': section.classification,
            'extraction_result_uri': getattr(section, 'extraction_result_uri', None),
            'has_results': hasattr(section, 'extraction_result_uri') and section.extraction_result_uri is not None
        } for section in (document.sections or [])
    ]
}

extraction_summary_path = data_dir / "extraction_summary.json"
with open(extraction_summary_path, 'w') as f:
    json.dump(extraction_summary, f, indent=2)

print(f"Saved document to: {document_path}")
print(f"Saved configuration to: {config_path}")
print(f"Saved environment info to: {env_path}")
print(f"Saved extraction summary to: {extraction_summary_path}")

### 6. Summary

In [ ]:
sections_processed = len(extraction_results) if 'extraction_results' in locals() else 0
sections_with_results = sum(1 for section in (document.sections or []) if hasattr(section, 'extraction_result_uri') and section.extraction_result_uri)

print("=== Step 3: Extraction Complete ===")
print(f"✅ Document processed: {document.id}")
print(f"✅ Sections processed: {sections_processed} of {len(document.sections) if document.sections else 0}")
print(f"✅ Sections with results: {sections_with_results}")
print(f"✅ Model used: {extraction_config.get('model')}")
print("✅ Data saved to: .data/step3_extraction/")
print("\n📌 Next step: Run step4_assessment.ipynb")

## Direct Comparison: Same Document, Both Methods

In [ ]:
# Let's extract from the same section using both methods for direct comparison
from copy import deepcopy


if document.sections and len(document.sections) > 0:
    test_section = document.sections[0]  # Use first section
    
    print(f"📋 Testing with Section: {test_section.classification}")
    print("="*60)
    
    # Track metrics
    comparison_results = {}
    
    # Method 1: Traditional Extraction
    print("\n🔴 METHOD 1: TRADITIONAL EXTRACTION")
    print("-"*40)
    
    CONFIG_TRAD = CONFIG.copy()
    CONFIG_TRAD["extraction"]["agentic"] = {"enabled": False}
    
    extraction_service_traditional = extraction.ExtractionService(config=CONFIG_TRAD)
    
    start_time = time.time()
    try:
        document_trad = extraction_service_traditional.process_document_section(
            document=deepcopy(document),
            section_id=test_section.section_id
        )
        trad_time = time.time() - start_time
        
        # Load results
        trad_section = document_trad.sections[0]
        if trad_section.extraction_result_uri:
            trad_data = load_json_from_s3(trad_section.extraction_result_uri)
            trad_result = trad_data.get('inference_result', {})
            
            comparison_results['traditional'] = {
                'time': trad_time,
                'fields_extracted': len([k for k, v in trad_result.items() if v is not None]),
                'total_fields': len(trad_result),
                'has_nested': any(isinstance(v, dict) for v in trad_result.values()),
                'has_arrays': any(isinstance(v, list) for v in trad_result.values())
            }
            
            print(f"✅ Completed in {trad_time:.2f} seconds")
            print(f"   Fields: {comparison_results['traditional']['fields_extracted']}/{comparison_results['traditional']['total_fields']}")
            print(f"   Complex structures: Nested={comparison_results['traditional']['has_nested']}, Arrays={comparison_results['traditional']['has_arrays']}")
    except Exception as e:
        print(f"❌ Failed: {e}")
        comparison_results['traditional'] = {'error': str(e)}
    
    # Method 2: Agentic Extraction
    print("\n🟢 METHOD 2: AGENTIC EXTRACTION")
    print("-"*40)
    
    CONFIG_AGENT = CONFIG.copy()
    CONFIG_AGENT["extraction"]["agentic"] = {"enabled": True}
    
    extraction_service_agentic = extraction.ExtractionService(config=CONFIG_AGENT)
    
    start_time = time.time()
    try:
        document_agent = extraction_service_agentic.process_document_section(
            document=deepcopy(document),
            section_id=test_section.section_id
        )
        agent_time = time.time() - start_time
        
        # Load results
        agent_section = document_agent.sections[0]
        if agent_section.extraction_result_uri:
            agent_data = load_json_from_s3(agent_section.extraction_result_uri)
            agent_result = agent_data.get('inference_result', {})
            
            comparison_results['agentic'] = {
                'time': agent_time,
                'fields_extracted': len([k for k, v in agent_result.items() if v is not None]),
                'total_fields': len(agent_result),
                'has_nested': any(isinstance(v, dict) for v in agent_result.values()),
                'has_arrays': any(isinstance(v, list) for v in agent_result.values())
            }
            
            print(f"✅ Completed in {agent_time:.2f} seconds")
            print(f"   Fields: {comparison_results['agentic']['fields_extracted']}/{comparison_results['agentic']['total_fields']}")
            print(f"   Complex structures: Nested={comparison_results['agentic']['has_nested']}, Arrays={comparison_results['agentic']['has_arrays']}")
    except Exception as e:
        print(f"❌ Failed: {e}")
        comparison_results['agentic'] = {'error': str(e)}
    
    # Comparison Summary
    print("\n📊 COMPARISON SUMMARY")
    print("="*60)
    
    if 'traditional' in comparison_results and 'agentic' in comparison_results:
        if 'error' not in comparison_results['traditional'] and 'error' not in comparison_results['agentic']:
            trad = comparison_results['traditional']
            agent = comparison_results['agentic']
            
            speed_diff = ((trad['time'] - agent['time']) / trad['time']) * 100
            field_diff = agent['fields_extracted'] - trad['fields_extracted']
            
            print(f"⏱️  Speed: Agentic is {speed_diff:.1f}% {'faster' if speed_diff > 0 else 'slower'}")
            print(f"    Traditional: {trad['time']:.2f}s | Agentic: {agent['time']:.2f}s")
            print()
            print(f"📝 Field Extraction: Agentic extracted {field_diff:+d} more fields")
            print(f"    Traditional: {trad['fields_extracted']}/{trad['total_fields']} | Agentic: {agent['fields_extracted']}/{agent['total_fields']}")
            print()
            print(f"🏗️  Structure Handling:")
            print(f"    Nested Objects: Traditional={trad['has_nested']} | Agentic={agent['has_nested']}")
            print(f"    Arrays: Traditional={trad['has_arrays']} | Agentic={agent['has_arrays']}")
            
            print("\n✨ KEY ADVANTAGES OF AGENTIC EXTRACTION:")
            advantages = []
            if speed_diff > 10:
                advantages.append(f"• {speed_diff:.0f}% faster processing")
            if field_diff > 0:
                advantages.append(f"• Better field coverage (+{field_diff} fields)")
            if agent['has_nested'] or agent['has_arrays']:
                advantages.append("• Handles complex nested structures")
            advantages.append("• Self-correcting with schema validation")
            advantages.append("• Consistent output format guaranteed")
            
            for adv in advantages:
                print(adv)
    else:
        print("⚠️ Comparison could not be completed due to errors")